# Confluence Module Docker Runner

# Run Confluence on LOCAL Machine

# Requirements
* docker installed somewhere where you have sudo priveledges to the point where "docker --version" completes successfully
* a dockerhub account (free)
* a python environment


# Overall Tasks
* Git clone all of the repos you want to run to a machine where you have sudo priveledges and where "docker --version" works (locally)
* Prep an empty_mnt directory to store confluence run (requires gdown package in environment)
* Run the "build_and_push_images" function of this notebook locally
* Run the "generate_run_scripts" section of this notebook to create python submission scripts for each module
* Run the generate_all_modules_bash Confluence Driver Script Generator section of this notebook  to create a .sh submission script that runs each of the modules one by one (the one click run)

In [ ]:
import os
import subprocess as sp

# FUNCTIONS IGNORE
def build_and_push_images(repo_directory:str, modules_to_run:list, docker_username:str, push:bool = True, custom_tag_name:str = 'latest'):
    for a_repo_name in modules_to_run:
        repo_path = os.path.join(repo_directory, a_repo_name)
        docker_path = f'{docker_username}/{a_repo_name}:{custom_tag_name}'
        build_cmd = ['docker', 'build','--quiet', '-f', os.path.join(repo_path, "Dockerfile"), '-t', docker_path, repo_path]
        try:
            sp.run(build_cmd)
        except Exception as e:
            raise RuntimeError(
                f"Docker build failed...\n"
                f"Build Command: {build_cmd}\n"
                f"Error: {e}"
            )
        if push:
            try:
                push_cmd = ['docker', 'push','--quiet', docker_path]
                sp.run(push_cmd)
            except Exception as e:
                raise RuntimeError(
                    f"Docker push failed...\n"
                    f"Push Command: {push_cmd}\n"
                    f"Error: {e}"
                )

import os
import json
import subprocess as sp

def generate_run_scripts(
    run,
    modules_to_run,          # list[str]
    script_jobs,             # dict of module -> job count (str), e.g., "7" or "$default_jobs"
    base_dir,
    repo_directory,
    rebuild_docker,
    docker_username,
    push,
    custom_tag_name,
    default_jobs_json_path=None
):
    """
    Generates a separate Python run script for each module in modules_to_run.
    If a module's job count is "$default_jobs", dynamically determine the number
    of jobs from the length of the JSON list at default_jobs_json_path.

    Parameters:
    - run (str): Name of the confluence run
    - modules_to_run (list[str]): Modules to generate scripts for.
    - script_jobs (dict): Module -> job count (int as str or "$default_jobs").
    - base_dir (str): Path to repos.
    - rebuild_docker (bool): Whether to rebuild docker images.
    - default_jobs_json_path (str): JSON file path to determine default_jobs length.
    """
    # Has to exist with 'mnt' structure (Doit exister avec la structure 'mnt')
    mnt_dir = os.path.join(base_dir, f'confluence_{run}', f'{run}_mnt')
    
    # Create the sh_scripts directory (Cree le repertoire sh_scripts)
    sh_dir = os.path.join(base_dir, f'confluence_{run}', 'sh_scripts')
    if not os.path.exists(sh_dir):
        os.makedirs(sh_dir)
    
    # Create the sif directory (Cree la repertoire sif)
    sif_dir = os.path.join(base_dir, f'confluence_{run}', 'sif')
    if not os.path.exists(sif_dir):
        os.makedirs(sif_dir)
    
    # Create the report directory (Cree la repertoire report)
    report_dir = os.path.join(base_dir, f'confluence_{run}', 'report')
    if not os.path.exists(report_dir):
        os.makedirs(report_dir)

    # This is a dictionary of all of the Confluence module run commands translated to singularity run commands.
    # You should not have to change anything here.
    command_dict = {
        'expanded_setfinder': f'docker run -v {mnt_dir}/input:/data setfinder -r reaches_of_interest.json -c continent.json -e -s 16 -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i index_to_run',
        'expanded_combine_data': f'docker run -v {mnt_dir}/input:/data combine_data -d /data -e -s 16',
        'input': f'docker run -v {mnt_dir}/input:/mnt/data input -r /mnt/data/expanded_reaches_of_interest.json -i index_to_run',
        'non_expanded_setfinder': f'docker run -v {mnt_dir}/input:/data setfinder -c continent.json -s 16 -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i index_to_run',
        'non_expanded_combine_data': f'docker run -v {mnt_dir}/input:/data combine_data -d /data -s 16',
        'prediagnostics': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/diagnostics/prediagnostics:/mnt/data/output prediagnostics -r reaches.json -i index_to_run',
        'unconstrained_priors': f'docker run -v {mnt_dir}/input:/mnt/data priors -r unconstrained -p usgs riggs -g -s local -i index_to_run', # Branch local_run
        'metroman': f'docker run --env AWS_BATCH_JOB_ID="foo" -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe/metroman:/mnt/data/output metroman -r metrosets.json -s local -v -i index_to_run', # branch local_run_args
        'metroman_consolidation': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe/metroman:/mnt/data/flpe metroman_consolidation -i index_to_run',
        'unconstrained_momma': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe/momma:/mnt/data/output momma -r reaches.json -m 3 -i index_to_run',
        'neobam': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe/geobam:/mnt/data/output neobam -r reaches.json -i index_to_run',
        'sad': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe/sad:/mnt/data/output sad --reachfile reaches.json --index index_to_run',
        'MOI': f'docker run --env AWS_BATCH_JOB_ID="foo" -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe:/mnt/data/flpe -v {mnt_dir}/moi:/mnt/data/output moi -j basin.json -v -b unconstrained -s local -i index_to_run',
        'unconstrained_offline': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe:/mnt/data/flpe -v {mnt_dir}/moi:/mnt/data/moi -v {mnt_dir}/offline:/mnt/data/output offline unconstrained timeseries integrator reaches.json index_to_run',
        'Validation': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe:/mnt/data/flpe -v {mnt_dir}/moi:/mnt/data/moi -v {mnt_dir}/offline:/mnt/data/offline -v {mnt_dir}/validation:/mnt/data/output validation reaches.json unconstrained index_to_run',
        'output': f'docker run -v {mnt_dir}/input:/mnt/data/input -v {mnt_dir}/flpe:/mnt/data/flpe -v {mnt_dir}/moi:/mnt/data/moi -v {mnt_dir}/diagnostics:/mnt/data/diagnostics -v {mnt_dir}/offline:/mnt/data/offline -v {mnt_dir}/validation:/mnt/data/validation -v {mnt_dir}/output:/mnt/data/output output -s local -j /app/metadata/metadata.json -m input priors prediagnostics momma neobam metroman sic4dvar sad moi offline validation swot -i index_to_run'
    }
    
    # Build docker images once if requested
    if rebuild_docker:
        build_and_push_images(
            repo_directory=repo_directory,
            modules_to_run=modules_to_run,
            docker_username=docker_username,
            push=push,
            custom_tag_name=custom_tag_name
        )

    # Load default_jobs from JSON if needed
    default_jobs = None
    if "$default_jobs" in script_jobs.values():
        if default_jobs_json_path is None:
            raise ValueError("default_jobs_json_path must be provided when using $default_jobs")
        with open(default_jobs_json_path, "r") as f:
            data = json.load(f)
            if isinstance(data, list):
                default_jobs = len(data)
            else:
                raise ValueError(f"Expected a JSON list at {default_jobs_json_path}")

    output_paths = []

    for module in modules_to_run:
        job_count = script_jobs.get(module, "1")
        if job_count == "$default_jobs":
            if default_jobs is None:
                raise ValueError(f"default_jobs JSON not loaded for module {module}")
            start_index = 0
            end_index = default_jobs - 1
        else:
            start_index = 0
            end_index = int(job_count) - 1

        script_content = f"""#!/usr/bin/env python3
import subprocess as sp

# Docker command for module: {module}
command = f'{command_dict[module]}'

for index in range({start_index}, {end_index} + 1):
    print(f"Running command for module '{module}' at index {{index}}")
    run_command = command.replace('index_to_run', str(index))
    sp.run(run_command, shell=True, check=True)
"""

        output_script_path = os.path.join(sh_dir, f"run_{module}.py")
        with open(output_script_path, 'w') as f:
            f.write(script_content)

        os.chmod(output_script_path, 0o755)
        output_paths.append(output_script_path)
        print(f"Python script created: {output_script_path}")

    return output_paths


import os

def generate_run_all_modules_bash(
    modules_to_run: list[str],
    script_jobs: dict,
    base_dir: str,
    script_folder_name: str = "sh_scripts",
    bash_script_name: str = "run_all_modules.sh"
) -> str:
    """
    Generates a Bash script that runs all per-module scripts in series.
    Only includes modules for which a script was actually generated.

    Parameters:
    - modules_to_run: list of module names
    - script_jobs: dict of module -> job count (str), used to filter modules
    - base_dir: directory to save the bash script
    - script_folder_name: folder inside base_dir where run_<module>.py scripts reside
    - bash_script_name: name of the generated bash script

    Returns:
    - full path to the generated bash script
    """
    os.makedirs(base_dir, exist_ok=True)
    bash_script_path = os.path.join(base_dir, f'confluence_{run}', bash_script_name)

    # Only include modules that have non-zero job counts
    filtered_modules = []
    for module in modules_to_run:
        count = script_jobs.get(module, "0")
        if count != "0":
            filtered_modules.append(module)

    # Generate modules array string
    modules_array_str = "modules_to_run=(\n"
    for module in filtered_modules:
        modules_array_str += f"    \"{module}\"\n"
    modules_array_str += ")\n"

    script_content = f"""#!/bin/bash
# {bash_script_name}
# Runs all generated module scripts in series

SCRIPT_DIR="{script_folder_name}"

{modules_array_str}

echo "Starting module runs..."

for module in "${{modules_to_run[@]}}"; do
    script_path="${{SCRIPT_DIR}}/run_${{module}}.py"

    if [[ -f "$script_path" ]]; then
        echo "---------------------------------------------"
        echo "Running module: $module"
        echo "Script: $script_path"
        echo "---------------------------------------------"
        python3 "$script_path"
        if [[ $? -ne 0 ]]; then
            echo "Error occurred while running $module. Exiting."
            exit 1
        fi
        echo "Finished module: $module"
        echo
    else
        echo "Script not found for module: $module. Skipping."
    fi
done

echo "All modules completed successfully."
"""

    with open(bash_script_path, "w") as f:
        f.write(script_content)

    os.chmod(bash_script_path, 0o755)
    print(f"Generated bash script: {bash_script_path}")
    return bash_script_path


In [ ]:

import os
import subprocess as sp
run = 'run1'

base_dir = '/Users/elisafriedmann/Library/CloudStorage/OneDrive-UniversityofMassachusetts/UMass/SWOT/confluence/run-confluence-locally/'  # Downloaded using: gdown 1xRltFZ1gyP_nvwHMJW-rIgClzXx8CSLC

repo_directory = '/home/travis/repos' # Contains the github cloned repos of the modules you want to run, check documentation for branches



script_jobs = {
        "expanded_setfinder": "7",
        "expanded_combine_data": "1",
        "input_so": "$default_jobs",
        "non_expanded_setfinder": "7",
        "non_expanded_combine_data": "1",
        "prediagnostics_permissive": "$default_jobs",
        # "unconstrained_priors": "7", 
        "sad": "$default_jobs",
        "metroman": "$default_jobs",
        "metroman_consolidation": "$default_jobs",
        "sic4dvar": "$default_jobs",
        "unconstrained_momma": "$default_jobs",
        "neobam": "$default_jobs",
        "moi": "$default_jobs",
        "unconstrained_offline": "$default_jobs",
        "validation": "$default_jobs",
        "output": "7",
    }


#------------------------------------------------

# SETUP

modules_to_run = ['expanded_setfinder', 'expanded_combine_data', 'input'] # Chose what module to run using the below command dict, they are listed in order. eg: run expanded_setfinder first

# Only provide this if you want to store images on dockerhub to move to HPC (you probably do)
push = True
docker_username = 'mydockerusername'
custom_tag_name = run # good to name same as the run, will default to 'latest'

# --------------------------------------------------------------------------------------


In [ ]:


# Generate Docker images from cloned modules

build_and_push_images(\
                      repo_directory = repo_directory, \
                      modules_to_run = modules_to_run, \
                      docker_username = docker_username, \
                      push = push, \
                      custom_tag_name = custom_tag_name \
                     )
                      
# The output should look something like 
# sha256:6900c3d99325a4a7c8b282d4a7a62f2a0f3fc673f03f5ca3333c2746bf20d06a
# docker.io/travissimmons/setfinder:latest

# Then, generate the Python script that runs the command
# Change Rebuild docker to TRUE if you made module and tag changes for a one-click run 

generate_run_scripts(
    run=run,
    modules_to_run=modules_to_run,
    script_jobs=script_jobs,
    base_dir=base_dir,
    repo_directory=repo_directory,
    rebuild_docker=False,
    docker_username=docker_username,
    push=False,
    custom_tag_name=run,
    default_jobs_json_path=os.path.join(base_dir, f"confluence_{run}", f"{run}_mnt", "input", "reaches_of_interest.json")
)

generate_run_all_modules_bash(
    modules_to_run=modules_to_run,
    script_jobs=script_jobs,
    base_dir=base_dir,
    script_folder_name = "sh_scripts",
    bash_script_name = "run_all_modules.sh"
)


In [ ]:
# After running this notebook, there will be individual python scripts and run_all_module.sh file generated in the same directory.
# You can either run the python scripts individually or run the .sh script to run them all on the command line
